# T Gate

In [2]:
from IPython.display import clear_output # ignore - just cleanup
!python -m pip install qiskit qiskit-aer qiskit_ibm_runtime pylatexenc matplotlib # essential libraries to have
clear_output() # ignore - just cleanup

In [ ]:
from LogicalQ.Logical import LogicalStatevector

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, partial_trace

from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Experiments import execute_circuits
from LogicalQ.Logical import LogicalStatevector
import numpy as np

from qiskit_aer import AerSimulator

# Generates the circuit
pqc = QuantumCircuit(1)
pqc.t(0)
lqcirc = LogicalCircuit.from_physical_circuit(pqc, **steane_code)

shots = 32
simulator = AerSimulator()

lqcirc.encode(0, max_iterations=1, initial_states=[1])

# Adds QEC cycle and measurement
#lqcirc.append_qec_cycle([0, 1])
lqcirc.measure([0], [0])
#lqcirc.measure_all()

# Logical Statevector
result = execute_circuits(lqcirc, backend=simulator, shots=shots, memory=True, save_statevector=False)[0]
counts = result.get_counts(lqcirc)
logical_statevector = LogicalStatevector.from_counts(counts, 1, **steane_code)




In [11]:
print("\nLogical Statevector:")
logical_statevector


Logical Statevector:
0j |0>_L + (0.976489860701969+0j) |1>_L + (0.02351013929803103+0j) |psi_L^perp>


In [14]:
# Exact statevector
qubits_to_trace_out = []
shots = 1
for i in range(shots):
    result = execute_circuits(lqcirc, backend=simulator, shots=1, memory=True, save_statevector=False)[0]
    statevector = Statevector(result.get_statevector())
    #target_qubit = partial_trace(statevector, qubits_to_trace_out)
    statevector.draw('latex', prefix='Statevector:')

QiskitError: 'No statevector for experiment "None"'

In [13]:
print("Actual Statevector:")
statevector.draw('latex', prefix='Statevector:')

Actual Statevector:


<IPython.core.display.Latex object>

# Scrapwork

In [ ]:
logical_statevec = LogicalStatevector(lqcirc, n_logical_qubits=1, )

In [ ]:
def tomography(lqcirc: LogicalCircuit):
    lqcirc_z = lqcirc.copy()
    lqcirc_x = lqcirc.copy()
    
    

In [ ]:
statevec

In [ ]:
lqcirc.draw(output="mpl")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

from LogicalQ.Library.QECCs import phase_flip_code
from LogicalQ.Experiments import execute_circuits
from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Analysis import counts_to_statevector

%load_ext autoreload
%autoreload 2

In [ ]:
T_gate = np.array([[1, 0], [0, np.exp(np.pi/4 * 1j)]])
init_state = np.array([0, 0]).T

final_state = T_gate @ init_state
final_state

In [ ]:
# Creating a Quantum Register with 1 Qubit
qr = QuantumRegister(1)

# Creating a Quantum Circuit
circuit = QuantumCircuit(qr)

# Applying T Gate on the first Qubit
circuit.t(0)

# Drawing the Quantum Circuit
circuit.draw()

In [ ]:
no_correction_code = lambda n_qubits : {
    "label": (n_qubits, n_qubits, 0),
    "stabilizer_tableau": [
        (1 * n_qubits) * "I"
    ] * 2 * n_qubits
}

In [ ]:
no_correction_code = lambda n_qubits : {
    "label": (n_qubits, n_qubits, 0),
    "stabilizer_tableau": []
}

code = no_correction_code(1)

lqc = LogicalCircuit(1, **code)
lqc.draw("mpl")

In [ ]:
# Runs a shot on a noiseless simulator with an error at each of the physical qubits

shots = 32
simulator = AerSimulator()

# Generates the circuit
pqc = QuantumCircuit(1)
pqc.t(0)
pqc.draw(output="mpl")
pqc.num_qubits

In [ ]:
lqcirc = LogicalCircuit.from_physical_circuit(pqc, **phase_flip_code)
lqcirc.draw(output="mpl")

In [ ]:
lqcirc.encode(0, max_iterations=1, initial_states=[0])

# Adds QEC cycle and measurement
#lqcirc.append_qec_cycle([0, 1])
#lqcirc.measure([0, 1], [0, 1])
lqcirc.measure_all()

# Simulates the QEC cycle
result = execute_circuits(lqcirc, backend=simulator, shots=shots, memory=True)[0]
counts = result.get_counts(lqcirc)
    
# Outputs should all be 10 if the error correction worked properly
#statevector = counts_to_statevector(counts)

In [ ]:
from LogicalQ.Logical import LogicalStatevector

statevec = LogicalStatevector.from_counts(counts, 2, **phase_flip_code)

In [ ]:
result.get_counts(lqcirc)

In [ ]:
result_key_0 = list(counts.keys())[2]
result_key_0

In [ ]:
result_key_0 = ["0b" + id for id in list(counts.keys())]
result_key_0

In [ ]:
# result_key_0 = list(counts.keys())[0]
#    if all([char in ["0", "1"] for char in result_key_0]):
#result_key_0 = list(counts.keys())[:][:2]
all([char in ["0", "1"] for char in result_key_0])

d = 2**(len(result_key_0))
fmt_outcome = lambda outcome : bin(outcome)[2:]

In [ ]:
d

In [ ]:
outcomes = [fmt_outcome(i) for i in range(d)]

In [ ]:
from qiskit.quantum_info import Statevector

outcomes = [fmt_outcome(i) for i in range(d)]

probabilities = np.array([counts.get(outcome, 0.0) for outcome in outcomes]) / np.sum(list(counts.values()))
amplitudes = np.sqrt(probabilities)

statevector = Statevector(amplitudes)

In [ ]:
statevector